<a href="https://colab.research.google.com/github/pranay8297/deep-learning-projects/blob/master/embedding_backprop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from fastai import *
from fastai.collab import *

In [2]:
!wget http://files.grouplens.org/datasets/movielens/ml-100k.zip

--2020-10-07 04:06:26--  http://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip’

ml-100k.zip         100%[===================>]   4.70M  5.62MB/s    in 0.8s    

2020-10-07 04:06:27 (5.62 MB/s) - ‘ml-100k.zip’ saved [4924029/4924029]



In [3]:
!pip install ipdb
from ipdb import set_trace as st

  Created wheel for ipdb: filename=ipdb-0.13.4-cp36-none-any.whl size=10973 sha256=bc23b38b5faafa99e1b947c68c76141de50d8b485a5b204b42f28807efa73c0b
  Stored in directory: /root/.cache/pip/wheels/56/51/e4/c91c61e3481a1a967beb18c4ea7a2b138a63cce94170b2e206
Successfully built ipdb


In [4]:
!ls

ml-100k.zip  sample_data


In [5]:
# unzip 'ml-100k.zip'
!mkdir data

In [6]:
!unzip ml-100k.zip -d data

Archive:  ml-100k.zip
   creating: data/ml-100k/
  inflating: data/ml-100k/allbut.pl  
  inflating: data/ml-100k/mku.sh     
  inflating: data/ml-100k/README     
  inflating: data/ml-100k/u.data     
  inflating: data/ml-100k/u.genre    
  inflating: data/ml-100k/u.info     
  inflating: data/ml-100k/u.item     
  inflating: data/ml-100k/u.occupation  
  inflating: data/ml-100k/u.user     
  inflating: data/ml-100k/u1.base    
  inflating: data/ml-100k/u1.test    
  inflating: data/ml-100k/u2.base    
  inflating: data/ml-100k/u2.test    
  inflating: data/ml-100k/u3.base    
  inflating: data/ml-100k/u3.test    
  inflating: data/ml-100k/u4.base    
  inflating: data/ml-100k/u4.test    
  inflating: data/ml-100k/u5.base    
  inflating: data/ml-100k/u5.test    
  inflating: data/ml-100k/ua.base    
  inflating: data/ml-100k/ua.test    
  inflating: data/ml-100k/ub.base    
  inflating: data/ml-100k/ub.test    


In [7]:
!cat data/ml-100k/README

SUMMARY & USAGE LICENSE

MovieLens data sets were collected by the GroupLens Research Project
at the University of Minnesota.
 
This data set consists of:
	* 100,000 ratings (1-5) from 943 users on 1682 movies. 
	* Each user has rated at least 20 movies. 
        * Simple demographic info for the users (age, gender, occupation, zip)

The data was collected through the MovieLens web site
(movielens.umn.edu) during the seven-month period from September 19th, 
1997 through April 22nd, 1998. This data has been cleaned up - users
who had less than 20 ratings or did not have complete demographic
information were removed from this data set. Detailed descriptions of
the data file can be found at the end of this file.

Neither the University of Minnesota nor any of the researchers
involved can guarantee the correctness of the data, its suitability
for any particular purpose, or the validity of results based on the
use of the data set.  The data set may be used for any research
purposes under th

In [ ]:
!pwd

/content


In [ ]:
!cd data

In [8]:
path = Path('/content')
path.ls()
path = path/'data'
path.ls()
path = path/'ml-100k'
path.ls()

[PosixPath('/content/data/ml-100k/README'),
 PosixPath('/content/data/ml-100k/u2.base'),
 PosixPath('/content/data/ml-100k/ub.base'),
 PosixPath('/content/data/ml-100k/u5.base'),
 PosixPath('/content/data/ml-100k/u5.test'),
 PosixPath('/content/data/ml-100k/u.genre'),
 PosixPath('/content/data/ml-100k/ub.test'),
 PosixPath('/content/data/ml-100k/ua.base'),
 PosixPath('/content/data/ml-100k/u.user'),
 PosixPath('/content/data/ml-100k/u4.test'),
 PosixPath('/content/data/ml-100k/ua.test'),
 PosixPath('/content/data/ml-100k/u.data'),
 PosixPath('/content/data/ml-100k/allbut.pl'),
 PosixPath('/content/data/ml-100k/u.info'),
 PosixPath('/content/data/ml-100k/mku.sh'),
 PosixPath('/content/data/ml-100k/u2.test'),
 PosixPath('/content/data/ml-100k/u1.base'),
 PosixPath('/content/data/ml-100k/u3.test'),
 PosixPath('/content/data/ml-100k/u.item'),
 PosixPath('/content/data/ml-100k/u4.base'),
 PosixPath('/content/data/ml-100k/u3.base'),
 PosixPath('/content/data/ml-100k/u1.test'),
 PosixPath('/c

In [12]:
movies = pd.read_csv(path/'u.item', encoding='latin-1', delimiter='|', header = None, names = ["movie_id", "movie_title", "release date", "video release date", "IMDb URL", "unknown", "Action", "Adventure", "Animation", "Children's", "Comedy", "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror", "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"])
movies.head()

,movie_id,movie_title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0


In [11]:
ratings = pd.read_csv(path/'u.data', delimiter='\t', header= None, names=['user_id', 'movie_id', 'rating', 'timestamp'])
ratings.head()

,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


,movie_id,movie_title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)
...,...,...
1677,1678,Mat' i syn (1997)
1678,1679,B. Monkey (1998)
1679,1680,Sliding Doors (1998)
1680,1681,You So Crazy (1994)


In [13]:
data_df = ratings.merge(movies[['movie_id', 'movie_title']], how = 'inner', on = 'movie_id')
data_df.head()

,user_id,movie_id,rating,timestamp,movie_title
0,196,242,3,881250949,Kolya (1996)
1,63,242,3,875747190,Kolya (1996)
2,226,242,5,883888671,Kolya (1996)
3,154,242,3,879138235,Kolya (1996)
4,306,242,5,876503793,Kolya (1996)


In [14]:
data = CollabDataBunch.from_df(data_df, item_name = 'movie_title', bs = 64)

In [35]:
# batch= data.one_batch()
# user, movie = batch[0]
# ratings = batch[1]
for batch in data.train_dl:
  break
user, movie = batch[0]
ratings = batch[1]

In [36]:
user.shape, movie.shape, ratings.shape, user

(torch.Size([64]),
 torch.Size([64]),
 torch.Size([64]),
 tensor([870, 610, 339, 409, 151, 497,  18, 328,  64, 825,  94, 130,   5,  56,
         739, 819, 320, 500, 303, 487, 499, 465, 394, 883, 711, 332, 222, 279,
         193, 280, 608, 222, 931,  92,  10, 495, 116, 527, 486, 591, 271, 660,
         768, 308, 934, 115, 188, 494, 851, 305, 605, 710, 382, 151, 716, 671,
         458, 707, 699, 655, 658, 128, 828, 896], device='cuda:0'))

In [92]:
#test
for i in data.train_ds.x.classes:
  pass

In [62]:
#test
# x = embedding(10, 20)
x = nn.Embedding(10, 20)

for i in x.parameters():
  # print(i)
  break

i.shape

torch.Size([10, 20])

In [51]:
#test
torch.tensor([1, 2])

tensor([1, 2])

In [60]:
#test
(x(torch.tensor([1, 2])) * x(torch.tensor([1, 2]))).sum(1)

tensor([30.6507, 10.7239], grad_fn=<SumBackward1>)

In [115]:
class MyEDB(nn.Module):
  def __init__(self, n_users, n_items, n_factors, y_range):
    super().__init__()
    self.y_range = y_range
    self.u_weight, self.i_weight, self.u_bias, self.i_bias = [MyEmbedding(ni, nf) for ni, nf in [(n_users, n_factors), (n_items, n_factors), (n_users, 1), (n_items, 1)]]
    # self.u_weight = nn.Parameter(MyEmbedding(n_users, n_factors))
    # self.i_weight = nn.Parameter(MyEmbedding(n_items, n_factors))
    # self.u_bias = nn.Parameter(MyEmbedding(n_users, 1))
    # self.i_bias = nn.Parameter(MyEmbedding(n_items, 1))
  
  def range_sigmoid(self, result):
    return torch.sigmoid(result) * (self.y_range[1] - self.y_range[0]) + self.y_range[0]
  
  def forward(self, users, items):

    weight_prod = self.u_weight(users) * self.i_weight(items)
    result = weight_prod.sum(1) + self.u_bias(users).squeeze(1) + self.i_bias(users).squeeze(1)
    final_result = self.range_sigmoid(result)
    return final_result


In [22]:
n, c = len(data.train_ds.x.classes['user_id']), len(data.train_ds.x.classes['movie_title'])

In [116]:
model = MyEDB(n, c, 40, (0, 5.5)).cuda()
model

MyEDB(
  (u_weight): MyEmbedding()
  (i_weight): MyEmbedding()
  (u_bias): MyEmbedding()
  (i_bias): MyEmbedding()
)

In [114]:
#test
# for i in model.parameters():
  # print(i)
u_m = MyEmbedding(n, 20).cuda()
i_m = MyEmbedding(c, 20).cuda()
# user.dtype
# user, user.long()
u_m(user), u_m(user).shape 
# u_m.emb.data

(tensor([[-0.5996, -0.9053,  0.8794,  ..., -0.1946,  1.3010,  0.5292],
         [-0.3972, -0.3926, -1.7937,  ..., -2.2752, -0.7389, -0.5389],
         [-0.3272,  1.0428,  2.6085,  ..., -0.7081,  0.0646, -1.3481],
         ...,
         [-0.4820, -0.0632,  2.1626,  ..., -0.2523,  0.5750, -2.2617],
         [ 0.4187,  2.1166,  0.6042,  ..., -1.4539, -0.9646,  0.6769],
         [-1.9393, -2.0254, -0.4942,  ..., -1.6524, -0.4844,  0.0117]],
        device='cuda:0', requires_grad=True), torch.Size([64, 20]))

In [117]:
res = model(user, movie)
res, res.shape

(tensor([5.4917e+00, 2.8517e+00, 5.1951e+00, 5.4902e+00, 6.6407e-06, 1.4152e-05,
         5.0447e-01, 2.3678e+00, 1.4374e-01, 2.9443e+00, 4.0001e+00, 5.4870e+00,
         5.4999e+00, 5.3608e+00, 5.4689e+00, 5.1080e-05, 5.0005e+00, 4.1981e+00,
         6.0866e-02, 1.3978e-01, 5.3826e+00, 3.2140e+00, 9.9974e-03, 9.0911e-04,
         5.9378e-02, 1.4787e+00, 2.4028e+00, 5.4983e+00, 5.4528e+00, 4.3283e-01,
         1.5223e-02, 5.1177e+00, 9.4617e-02, 1.2275e+00, 2.5054e-01, 5.5000e+00,
         5.4044e+00, 3.0433e-03, 5.3529e-02, 1.9884e-01, 4.0851e+00, 1.9852e+00,
         8.1263e-02, 9.8860e-02, 6.2038e-03, 5.4775e+00, 5.1990e+00, 2.3986e-02,
         5.1960e+00, 5.6188e-01, 5.4995e+00, 1.4295e-01, 2.6163e-01, 1.2517e-01,
         1.1813e+00, 5.5000e+00, 5.4837e+00, 1.2060e+00, 4.7117e-03, 3.5385e+00,
         4.7665e+00, 9.0456e-03, 5.4901e+00, 5.4915e+00], device='cuda:0',
        grad_fn=<AddBackward0>), torch.Size([64]))

In [118]:
def loss_function(y_hat, y):
  mse_loss = (y_hat - y)**2
  return mse_loss.sqrt().sum()/mse_loss.shape[0]

In [119]:
# ratings.cuda(), res
loss = loss_function(res, ratings.cuda())
loss
loss.backward()

In [130]:
for param in model.parameters():
  break

# param, param.grad
model.u_weight.emb

In [63]:
#test
a = torch.randn((10, 5), requires_grad=True).cuda()
a, a[torch.tensor([0, 1, 2])]
# torch.tensor([0, 1, 2]).dtype

(tensor([[ 0.2933,  0.1678,  1.6493,  0.5070, -1.1537],
         [ 0.5311,  2.0280,  0.0223,  0.1373,  0.3148],
         [-0.4039,  1.6639, -0.2573,  0.1303, -1.3362],
         [ 0.1399,  0.3028,  0.7724,  1.7914, -0.7841],
         [-1.0281, -1.0424,  1.5089,  0.7150,  0.1938],
         [ 1.0600,  1.9675, -1.8780, -0.2673, -2.4966],
         [ 0.6604,  0.3189, -1.1199,  0.5412,  0.2762],
         [-1.2190,  0.5791, -1.7207, -0.5075,  0.8427],
         [-0.4236, -1.3197,  0.6387, -0.9594, -0.9969],
         [ 0.3817, -1.1524, -0.0973,  1.6866,  1.3982]], device='cuda:0',
        grad_fn=<CopyBackwards>),
 tensor([[ 0.2933,  0.1678,  1.6493,  0.5070, -1.1537],
         [ 0.5311,  2.0280,  0.0223,  0.1373,  0.3148],
         [-0.4039,  1.6639, -0.2573,  0.1303, -1.3362]], device='cuda:0',
        grad_fn=<IndexBackward>))

In [112]:
class MyEmbedding(nn.Module):
  def __init__(self, ni, nf):
    super().__init__()
    self.emb = nn.Parameter(torch.randn((ni, nf), requires_grad = True))
    # self.emb = torch.randn((ni, nf), requires_grad = True)
  
  def forward(self, indecis):
    desired = self.emb.data[indecis]
    desired.requires_grad = True
    return desired
    